# Get sample reports

For LATINX in AI workshop

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from pycocoevalcap.bleu import bleu_scorer
from pycocoevalcap.rouge import rouge

In [ ]:
%run ../datasets/common.py
%run ../datasets/iu_xray.py
%run ../utils/nlp.py
%run ../utils/__init__.py

In [ ]:
CONSTANT_REPORT = """the heart is normal in size . the mediastinum is unremarkable . 
the lungs are clear .
there is no pneumothorax or pleural effusion . no focal airspace disease .
no pleural effusion or pneumothorax ."""

In [ ]:
dataset = IUXRayDataset(dataset_type='all')
report_reader = ReportReader(dataset.get_vocab())
len(dataset)

In [ ]:
idx = GT_IDX
item = dataset[idx]
image = arr_to_range(item.image.permute(1, 2, 0))
report_base = report_reader.idx_to_text(item.report)
plt.imshow(image)
plt.axis('off')
print(report_base)

In [ ]:
GT_IDX = 7289

In [ ]:
target = [
    'the cardiac silhouette is enlarged',
    # 'the lungs are hyper',
    # 'the heart is',
]
not_target = [
    # 'the lungs are clear',
#     'the mediastinum is unremarkable',
#     'the mediastinum is stable',
#     'the mediastinum is normal',
#     'the mediastinum is within normal limits',
]
found = []
found_names = set()
for idx, report in enumerate(dataset.reports):
    filename = report['filename']
    report = report_reader.idx_to_text(report['tokens_idxs'])
    if all(t in report for t in target) and all(t not in report for t in not_target):
        if filename not in found_names:
            found.append((idx, report))
        found_names.add(filename)
len(found)

In [ ]:
found[5]

In [ ]:
gen = 'the heart is enlarged. the mediastinum is unremarkable . the lungs are hyperinflated with mildly coarsened interstitial markings . '
# the lungs are hyperexpanded
# the lungs are hyperinflated with mildly coarsened interstitial markings
# the lungs are hyperinflated with biapical pleural-parenchymal scarring and upward retraction of the xxxx

In [ ]:
def measure_bleu_rouge(gen, gt):
    scorer = bleu_scorer.BleuScorer(n=4)
    scorer += (gen, [gt])
    bleu_1_4, _ = scorer.compute_score()
    
    scorer = rouge.Rouge()
    rouge_score = scorer.calc_score([gen], [gt])
    
    print('BLEU 1-4: ', bleu_1_4)
    print('BLEU: ', np.mean(bleu_1_4))
    print('ROUGE-L: ', rouge_score)

In [ ]:
report_1 = """the heart is normal in size . the mediastinum is unremarkable . 
the lungs are clear ."""
report_2 = """the heart is normal . the mediastinum is otherwise unremarkable . 
lungs are both clear ."""
measure_bleu_rouge(report_1, report_2)

In [ ]:
report = report_reader.idx_to_text(dataset[GT_IDX].report)
report

In [ ]:
gt = """the cardiac silhouette is enlarged .
the lungs are hyperexpanded with flattening of the bilateral hemidiaphragms .
no pneumothorax or pleural effusion ."""
# the lungs are hyperinflated with mildly coarsened interstitial markings .
# with flattening of the bilateral hemidiaphragms 

In [ ]:
gen = """the cardiac silhouette is normal in size .
the lungs are clear .
no pneumothorax or pleural effusion ."""

In [ ]:
measure_bleu_rouge(gen, gt)

In [ ]:
gt = "the cardiac silhouette is enlarged . the lungs are hyperexpanded with flattening of the bilateral hemidiaphragms . no pneumothorax or pleural effusion ."
gen = "the cardiac silhouette is normal in size and configuration . the lungs are clear . no pneumothorax or pleural effusion ."
measure_bleu_rouge(gen, gt)

In [ ]:
measure_bleu_rouge(gen, gt)